### Feature based classification

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
from scipy.signal import find_peaks

In [1]:
# Load data
x=np.load('scaled_spec_resampled_array.npy')
y=np.load('labels_array.npy')-1

print(x.shape)
print(y.shape)

num_samples, height, width = x.shape

(1754, 2048, 80)
(1754,)


In [36]:
nfft = 2048
fs = 128000
f_lo = -64000.0
f_hi = 63937.5
noise_threshold = -83  # dBm (based on the paper's noise threshold)

freq_axis=np.linspace(f_lo, f_hi, nfft)
print(len(freq_axis))

2048


Denoising and feature extraction based on this paper:

Y. Kim and H. Ling, "Human Activity Classification Based on Micro-Doppler Signatures Using a Support Vector Machine," in IEEE Transactions on Geoscience and Remote Sensing, vol. 47, no. 5, pp. 1328-1337, May 2009

In [44]:
def denoise_and_extract_features(frequencies, spectrogram, noise_threshold):
    # Initialize feature variables
    torso_doppler_frequency = []
    total_bw_doppler = []
    offset_total_doppler = []
    bw_without_micro_dopplers = []
    normalized_std_doppler = []
    high_envelope = []
    low_envelope = []

    spec_denoised = np.where(spectrogram < noise_threshold, noise_threshold, spectrogram)
    for t_index in range(spec_denoised.shape[1]):

        column = spec_denoised[:, t_index]
        
        if np.all(column == noise_threshold):
            high_freq = 0
            low_freq = 0
            peak_freq = 0
            total_bw = 0
            offset_total = 0
            normalized_std = 0
        else:
            high_freq = frequencies[column > noise_threshold][-1]
            low_freq = frequencies[column > noise_threshold][0]
            # Torso Doppler Frequency (1)
            peak_freq = frequencies[np.argmax(column)]
            # Total Bandwidth of the Doppler Signal (2)
            total_bw = high_freq-low_freq  # Peak-to-peak value
            # Offset of the Total Doppler (3)
            offset_total = (high_freq + low_freq) / 2
            # Normalized STD of the Doppler Signal Strength (5)
            std_doppler = np.std(column[column > noise_threshold])
            mean_doppler = np.mean(column[column > noise_threshold])
            normalized_std = std_doppler / mean_doppler if mean_doppler != 0 else 0

        high_envelope.append(high_freq)
        low_envelope.append(low_freq)
        torso_doppler_frequency.append(peak_freq)
        total_bw_doppler.append(total_bw)
        offset_total_doppler.append(offset_total)
        normalized_std_doppler.append(normalized_std)

    # Bandwidth Without Micro-Dopplers (4)
    bw_without_micro_dopplers = np.mean(np.array(sorted(high_envelope)[-5:]) - np.array(sorted(low_envelope)[:5]))

    # Period of Limb Motion (6)
    peaks, _ = find_peaks(high_envelope, height=np.nanmean(high_envelope))
    peak_intervals = np.diff(peaks)
    periods = peak_intervals / fs
    mean_period = np.mean(periods) if len(periods) > 0 else 0

    return [np.mean(torso_doppler_frequency),
            np.mean(total_bw_doppler),
            np.mean(offset_total_doppler),
            bw_without_micro_dopplers,
            np.mean(normalized_std_doppler),
            mean_period]

In [45]:
# Extract features from the training set
features = []
for i in range(num_samples):
    features.append(denoise_and_extract_features(freq_axis, x[i,:,:], noise_threshold))
features = np.array(features)
print(features.shape)

(1754, 6)


In [46]:
x_train, x_test, y_train, y_test = train_test_split(features,y,test_size=0.2,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1403, 6)
(351, 6)
(1403,)
(351,)


In [47]:
# KNN
for k in np.arange(1,11):
    knn=KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train,y_train)
    y_pred=knn.predict(x_test)
    print('k:',k,'accuracy:',accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred, target_names=['Walking', 'Sitting Down', 'Standing Up', 'Picking up an Object', 'Drinking Water', 'Falling']))

k: 1 accuracy: 0.7663817663817664
k: 2 accuracy: 0.7635327635327636
k: 3 accuracy: 0.7834757834757835
k: 4 accuracy: 0.8034188034188035
k: 5 accuracy: 0.7891737891737892
k: 6 accuracy: 0.7948717948717948
k: 7 accuracy: 0.8091168091168092
k: 8 accuracy: 0.8034188034188035
k: 9 accuracy: 0.7977207977207977
k: 10 accuracy: 0.7948717948717948
                      precision    recall  f1-score   support

             Walking       1.00      0.99      0.99        70
        Sitting Down       0.84      0.87      0.85        54
         Standing Up       0.88      0.88      0.88        66
Picking up an Object       0.61      0.75      0.68        61
      Drinking Water       0.60      0.46      0.52        65
             Falling       0.83      0.83      0.83        35

            accuracy                           0.79       351
           macro avg       0.79      0.80      0.79       351
        weighted avg       0.79      0.79      0.79       351



In [48]:
# Random Forest
for trees in [10,100,250,500,1000]:
    rf=RandomForestClassifier(n_estimators=trees)
    rf.fit(x_train,y_train)
    y_pred=rf.predict(x_test)
    print('trees:',trees,'accuracy:',accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred, target_names=['Walking', 'Sitting Down', 'Standing Up', 'Picking up an Object', 'Drinking Water', 'Falling']))

trees: 10 accuracy: 0.8205128205128205
trees: 100 accuracy: 0.8575498575498576
trees: 250 accuracy: 0.8547008547008547
trees: 500 accuracy: 0.8575498575498576
trees: 1000 accuracy: 0.8575498575498576
                      precision    recall  f1-score   support

             Walking       0.99      0.96      0.97        70
        Sitting Down       0.93      0.96      0.95        54
         Standing Up       0.86      0.97      0.91        66
Picking up an Object       0.70      0.74      0.72        61
      Drinking Water       0.72      0.65      0.68        65
             Falling       1.00      0.89      0.94        35

            accuracy                           0.86       351
           macro avg       0.87      0.86      0.86       351
        weighted avg       0.86      0.86      0.86       351



In [49]:
# XGBoost
xgb=XGBClassifier(n_estimators=250)
xgb.fit(x_train,y_train)
y_pred=xgb.predict(x_test)
print('XGBoost accuracy:',accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred, target_names=['Walking', 'Sitting Down', 'Standing Up', 'Picking up an Object', 'Drinking Water', 'Falling']))

XGBoost accuracy: 0.8376068376068376
                      precision    recall  f1-score   support

             Walking       1.00      0.96      0.98        70
        Sitting Down       0.92      1.00      0.96        54
         Standing Up       0.88      0.95      0.91        66
Picking up an Object       0.64      0.70      0.67        61
      Drinking Water       0.67      0.55      0.61        65
             Falling       0.97      0.89      0.93        35

            accuracy                           0.84       351
           macro avg       0.84      0.84      0.84       351
        weighted avg       0.84      0.84      0.83       351



In [50]:
# SVM
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

svc = SVC(kernel='rbf', random_state=42)
svc.fit(x_train_scaled, y_train)

y_pred = svc.predict(x_test_scaled)
print('SVM accuracy:',accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred, target_names=['Walking', 'Sitting Down', 'Standing Up', 'Picking up an Object', 'Drinking Water', 'Falling']))

SVM accuracy: 0.8290598290598291
                      precision    recall  f1-score   support

             Walking       1.00      0.93      0.96        70
        Sitting Down       0.90      0.96      0.93        54
         Standing Up       0.90      0.95      0.93        66
Picking up an Object       0.62      0.64      0.63        61
      Drinking Water       0.65      0.62      0.63        65
             Falling       0.97      0.91      0.94        35

            accuracy                           0.83       351
           macro avg       0.84      0.84      0.84       351
        weighted avg       0.83      0.83      0.83       351

